In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from imblearn.over_sampling import SMOTE

In [2]:
# ==============================
# CONFIGURACIÓN GLOBAL
# ==============================
RANDOM_SEED = 42
TEST_SIZE = 0.3
N_FEATURES = 10  # número de características a seleccionar
np.random.seed(RANDOM_SEED)

# ==============================
# 1. CARGA DE DATOS
# ==============================
df = pd.read_csv("creditcard.csv")

print("="*60)
print("ANÁLISIS EXPLORATORIO INICIAL")
print("="*60)
print(f"Shape original del dataset: {df.shape}")
print(f"Número de transacciones totales: {len(df):,}")
print(f"Número de fraudes: {df['Class'].sum():,}")
print(f"\nDistribución de clases:")
print(df["Class"].value_counts())
print(f"\nPorcentaje de fraudes: {df['Class'].mean()*100:.3f}%")
print(f"Ratio de desbalance: {(df['Class']==0).sum()/(df['Class']==1).sum():.0f}:1")

ANÁLISIS EXPLORATORIO INICIAL
Shape original del dataset: (284807, 31)
Número de transacciones totales: 284,807
Número de fraudes: 492

Distribución de clases:
Class
0    284315
1       492
Name: count, dtype: int64

Porcentaje de fraudes: 0.173%
Ratio de desbalance: 578:1


In [3]:
# ==============================
# 2. PREPROCESAMIENTO
# ==============================
print("\n" + "="*60)
print("PREPROCESAMIENTO")
print("="*60)

# Separar variables predictoras y variable objetivo
X = df.drop(columns=["Class"])
y = df["Class"]

# Escalado de todas las variables numéricas
print("\n1. Aplicando StandardScaler a todas las características...")
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
print(f"   ✓ {X_scaled.shape[1]} características escaladas (media=0, std=1)")

# División en train/test (70/30)
print("\n2. División train/test (70%/30%) con estratificación...")
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_SEED
)

print(f"   ✓ Train: {X_train.shape[0]:,} muestras")
print(f"   ✓ Test:  {X_test.shape[0]:,} muestras")
print(f"\nDistribución en conjunto de entrenamiento:")
print(f"  - Clase 0 (Normal): {(y_train==0).sum():,} ({(y_train==0).mean()*100:.2f}%)")
print(f"  - Clase 1 (Fraude): {(y_train==1).sum():,} ({(y_train==1).mean()*100:.2f}%)")
print(f"  - Ratio de desbalance: {(y_train==0).sum()/(y_train==1).sum():.0f}:1")


PREPROCESAMIENTO

1. Aplicando StandardScaler a todas las características...
   ✓ 30 características escaladas (media=0, std=1)

2. División train/test (70%/30%) con estratificación...
   ✓ Train: 199,364 muestras
   ✓ Test:  85,443 muestras

Distribución en conjunto de entrenamiento:
  - Clase 0 (Normal): 199,020 (99.83%)
  - Clase 1 (Fraude): 344 (0.17%)
  - Ratio de desbalance: 579:1


In [4]:
# ==============================
# 3. SELECCIÓN DE CARACTERÍSTICAS (SelectKBest)
# ==============================
print("\n" + "="*60)
print("SELECCIÓN DE CARACTERÍSTICAS")
print("="*60)
print(f"\n3. Aplicando SelectKBest con Mutual Information...")
print(f"   Técnica: Selección basada en tests estadísticos")
print(f"   Objetivo: Seleccionar las {N_FEATURES} características más predictivas")
print(f"   Métrica: Información Mutua (detecta relaciones no lineales)")
print(f"   Ventaja: Muy eficiente computacionalmente")
print(f"   IMPORTANTE: Se aplica ANTES del balanceo para evitar sesgo")
print("\n   ⏱️  Este proceso toma solo unos segundos...\n")

# SelectKBest con información mutua (mejor para clasificación)
# NOTA CRÍTICA: Se ajusta sobre datos SIN balancear para selección objetiva
selector = SelectKBest(score_func=mutual_info_classif, k=N_FEATURES)
selector.fit(X_train, y_train)

# Obtener características seleccionadas
selected_features = X_train.columns[selector.get_support()].tolist()

# Mostrar scores de las características seleccionadas
scores = pd.DataFrame({
    'Feature': X_train.columns,
    'Score': selector.scores_
}).sort_values('Score', ascending=False)

print("\n" + "="*60)
print("VECTOR DE CARACTERÍSTICAS SELECCIONADAS")
print("="*60)
for i, feature in enumerate(selected_features, start=1):
    score = scores[scores['Feature'] == feature]['Score'].values[0]
    print(f"  {i:2d}. {feature:6s}  (Score: {score:.4f})")


SELECCIÓN DE CARACTERÍSTICAS

3. Aplicando SelectKBest con Mutual Information...
   Técnica: Selección basada en tests estadísticos
   Objetivo: Seleccionar las 10 características más predictivas
   Métrica: Información Mutua (detecta relaciones no lineales)
   Ventaja: Muy eficiente computacionalmente
   IMPORTANTE: Se aplica ANTES del balanceo para evitar sesgo

   ⏱️  Este proceso toma solo unos segundos...


VECTOR DE CARACTERÍSTICAS SELECCIONADAS
   1. V3      (Score: 0.0049)
   2. V4      (Score: 0.0053)
   3. V9      (Score: 0.0045)
   4. V10     (Score: 0.0077)
   5. V11     (Score: 0.0069)
   6. V12     (Score: 0.0077)
   7. V14     (Score: 0.0083)
   8. V16     (Score: 0.0062)
   9. V17     (Score: 0.0084)
  10. V18     (Score: 0.0043)


In [5]:
# ==============================
# 4. BALANCEO DE CLASES (solo sobre entrenamiento con características seleccionadas)
# ==============================
print("\n" + "="*60)
print("ESTRATEGIA DE BALANCEO")
print("="*60)
print("\n4. Aplicando SMOTE (Synthetic Minority Over-sampling Technique)...")
print("   Estrategia: Sobremuestreo de la clase minoritaria (fraudes)")
print("   Nota: Se aplica DESPUÉS de selección de características")
print("   Nota: Se aplica SOLO sobre entrenamiento para evitar data leakage\n")

smote = SMOTE(random_state=RANDOM_SEED)
X_train_bal, y_train_bal = smote.fit_resample(X_train[selected_features], y_train)

print("Distribución DESPUÉS del balanceo:")
print(f"  - Clase 0 (Normal): {(y_train_bal==0).sum():,} ({(y_train_bal==0).mean()*100:.2f}%)")
print(f"  - Clase 1 (Fraude): {(y_train_bal==1).sum():,} ({(y_train_bal==1).mean()*100:.2f}%)")
print(f"  - Nuevo ratio: {(y_train_bal==0).sum()/(y_train_bal==1).sum():.1f}:1")
print("   ✓ Dataset balanceado correctamente")



ESTRATEGIA DE BALANCEO

4. Aplicando SMOTE (Synthetic Minority Over-sampling Technique)...
   Estrategia: Sobremuestreo de la clase minoritaria (fraudes)
   Nota: Se aplica DESPUÉS de selección de características
   Nota: Se aplica SOLO sobre entrenamiento para evitar data leakage

Distribución DESPUÉS del balanceo:
  - Clase 0 (Normal): 199,020 (50.00%)
  - Clase 1 (Fraude): 199,020 (50.00%)
  - Nuevo ratio: 1.0:1
   ✓ Dataset balanceado correctamente


In [6]:
# ==============================
# 5. GUARDAR RESULTADOS
# ==============================
print("\n" + "="*60)
print("RESUMEN Y GUARDADO DE ARCHIVOS")
print("="*60)

print(f"\n✓ Escalado:          StandardScaler aplicado")
print(f"✓ División:          70% train ({X_train.shape[0]:,}) / 30% test ({X_test.shape[0]:,})")
print(f"✓ Selección:         SelectKBest con Mutual Information")
print(f"✓ Características:   {N_FEATURES} seleccionadas de {X_scaled.shape[1]} originales")
print(f"✓ Balanceo:          SMOTE aplicado DESPUÉS de selección")

print(f"\nDimensiones finales:")
print(f"  - X_train_bal: {X_train_bal.shape}")
print(f"  - y_train_bal: {y_train_bal.shape}")
print(f"  - X_test:      {X_test[selected_features].shape}")
print(f"  - y_test:      {y_test.shape}")

pd.Series(selected_features).to_csv("selected_features.csv", index=False, header=False)
X_train_bal.to_csv("X_train_bal.csv", index=False)
y_train_bal.to_csv("y_train_bal.csv", index=False, header=True)
X_test[selected_features].to_csv("X_test.csv", index=False)
y_test.to_csv("y_test.csv", index=False, header=True)

print("\n✓ Archivos guardados exitosamente:")
print("  - selected_features.csv")
print("  - X_train_bal.csv / y_train_bal.csv")
print("  - X_test.csv / y_test.csv")

print("\n" + "="*60)
print("✅ PREPROCESAMIENTO COMPLETADO CORRECTAMENTE")
print("="*60)


RESUMEN Y GUARDADO DE ARCHIVOS

✓ Escalado:          StandardScaler aplicado
✓ División:          70% train (199,364) / 30% test (85,443)
✓ Selección:         SelectKBest con Mutual Information
✓ Características:   10 seleccionadas de 30 originales
✓ Balanceo:          SMOTE aplicado DESPUÉS de selección

Dimensiones finales:
  - X_train_bal: (398040, 10)
  - y_train_bal: (398040,)
  - X_test:      (85443, 10)
  - y_test:      (85443,)

✓ Archivos guardados exitosamente:
  - selected_features.csv
  - X_train_bal.csv / y_train_bal.csv
  - X_test.csv / y_test.csv

✅ PREPROCESAMIENTO COMPLETADO CORRECTAMENTE
